# fasterrcnn_resnet50_fpn的官方使用介绍：
[fasterrcnn_resnet50_fpn使用介绍](https://pytorch.org/vision/stable/models/generated/torchvision.models.detection.fasterrcnn_resnet50_fpn.html)

In [1]:
import torch
import torchvision

import cv2 as cv

In [2]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(
    pretrained=True,
    weights=torchvision.models.detection.FasterRCNN_ResNet50_FPN_Weights.COCO_V1
)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
model.to(device)

cuda


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [3]:
coco_labels_name = ["unlabeled", "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat",
                    "traffic light", "fire hydrant", "street sign", "stop sign", "parking meter", "bench", "bird",
                    "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "hat", "backpack",
                    "umbrella", "shoe", "eye glasses", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard",
                    "sports_ball", "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket",
                    "bottle", "plate", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple",
                    "sandwich", "orange", "broccoli", "carrot", "hot_dog", "pizza", "donut", "cake", "chair", "couch",
                    "potted plant", "bed", "mirror", "dining table", "window", "desk", "toilet", "door", "tv", "laptop",
                    "mouse", "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator",
                    "blender", "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush",
                    "hair brush"]

In [4]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
])

In [5]:
def save_threshold_item(pred, threshold):
    """
    保留置信度>thresh_hold的pred

    :param pred: 模型输出
    :param threshold: 阈值
    :return: 返回截取后的pred
    """
    threshold_index = 0
    for i in range(len(pred)):
        for j, score in enumerate(pred[i]['scores']):
            if score <= threshold:
                threshold_index = j
                break
        pred[i]['boxes'] = pred[i]['boxes'][:threshold_index]
        pred[i]['labels'] = pred[i]['labels'][:threshold_index]
        pred[i]['scores'] = pred[i]['scores'][:threshold_index]

    return pred

In [6]:
def plot(pred, images):
    """
    在图像上绘制box，label，confidence
    :param pred: 模型输出
    :param image: opencv读取的图像，可以用列表装入多张图像
    :return:
    """

    for i, image in enumerate(images):

        boxes = pred[i]['boxes']
        labels = pred[i]['labels']
        scores = pred[i]['scores']
        for i, item in enumerate(zip(boxes, labels, scores)):
            xmin = int(item[0][0])
            ymin = int(item[0][1])
            xmax = int(item[0][2])
            ymax = int(item[0][3])

            # 绘制box
            cv.rectangle(img=image, pt1=[xmin, ymin], pt2=[xmax, ymax], color=[255, 0, 0], thickness=2)

            # 取出置信度
            confidence = item[2].cpu().detach().clone().numpy()
            confidence = round(float(confidence), 2)
            text = coco_labels_name[int(item[1].cpu().numpy())] + " " + str(confidence)
            font = cv.FONT_HERSHEY_SIMPLEX
            cv.putText(image, text, (xmin, ymin), font, 0.8, (0, 0, 255), 1, cv.LINE_AA)

    return images

In [ ]:
model.eval()

cap = cv.VideoCapture(0)

if not cap.isOpened():
    print("Cannot open camera")
    exit()

# 设置摄像头大小
cap.set(3, 800)
cap.set(4, 500)

while True:
    # 逐帧捕获
    ret, frame = cap.read()
    # 如果正确读取帧，ret为True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    # 转化为张量类型，归一化到[0...1]区间
    tensor_frame = transform(frame)

    # 将[C, H, W]转化为[1, C, H, W]
    depth = frame.shape[2]
    width = frame.shape[0]
    height = frame.shape[1]
    tensor_frame = torch.reshape(tensor_frame, (1, depth, width, height))
    tensor_frame = tensor_frame.to(device)

    # 得到模型输出
    pred = model(tensor_frame)

    # 对pred进行截断，保留confidence > threshold的部分
    threshold = 0.5
    pred = save_threshold_item(pred, threshold)

    # 在图像上绘制box，label，confidence
    [frame] = plot(pred, [frame])

    # 显示结果帧
    cv.imshow('frame', frame)
    if cv.waitKey(1) == ord('q'):
        break

# 完成所有操作后，释放捕获器
cap.release()
cv.destroyAllWindows()

QObject::moveToThread: Current thread (0xe9d067e0) is not the object's thread (0xe9c10400).
Cannot move to target thread (0xe9d067e0)

QObject::moveToThread: Current thread (0xe9d067e0) is not the object's thread (0xe9c10400).
Cannot move to target thread (0xe9d067e0)

QObject::moveToThread: Current thread (0xe9d067e0) is not the object's thread (0xe9c10400).
Cannot move to target thread (0xe9d067e0)

QObject::moveToThread: Current thread (0xe9d067e0) is not the object's thread (0xe9c10400).
Cannot move to target thread (0xe9d067e0)

QObject::moveToThread: Current thread (0xe9d067e0) is not the object's thread (0xe9c10400).
Cannot move to target thread (0xe9d067e0)

QObject::moveToThread: Current thread (0xe9d067e0) is not the object's thread (0xe9c10400).
Cannot move to target thread (0xe9d067e0)

QObject::moveToThread: Current thread (0xe9d067e0) is not the object's thread (0xe9c10400).
Cannot move to target thread (0xe9d067e0)

QObject::moveToThread: Current thread (0xe9d067e0) is n